# （雑に拡張した）Bradley-TerryモデルでMリーガーの強さを推定する

**このノートブックでやること**：1対1で行うゲームのプレイヤの強さを推定するために使われるBradley-Terryモデルを、麻雀用に拡張して、Mリーガーの強さを対局結果（NOT打牌内容）から統計的に推測します。

<span style="color:red">**注意**</span>：このノートブックで示されるMリーガーの強さは、対局の結果・スコアから（雑に拡張した）**Bradley-Terryモデルで推定**されたものです。
この記事で示された強さは、あくまで推定したものであり、また、モデルが異なれば結果も変わる、ということにご注意ください（つまり、このノートブックで示される強さが唯一絶対の正しい強さの値だ！**ということではありません**）。

## 3行でまとめる
 - 対局結果をもとにMリーガーの強さを推定する。Bradley-Terryモデルを使う。
 - 平均スコアで強さを推定した場合、「上手いプレイヤとの対戦割合が大きいプレイヤ」は不利に、上手いプレイヤとの対戦割合が小さいプレイヤ」は有利になるが、この方法ではそこらへんをうまくやる（きっと）。
 - 伊達朱里紗プロ強し（~2022/03/16までの結果を利用した場合）。

## Bradley-Terryモデル
理屈なんてどうでも良い、結果だけ知りたい、という方は[BTR<sup>4</sup>モデルの推定 - 順位点も込み](#btr4_estimation_with_rank_point)まで飛んでください。
### 問題設定
1対1で行うゲームの、$M$人のプレイヤが行った、$N$回の対戦結果の集合があるとします。
対戦結果としては、「どちらが勝ったか（or 負けたか）」が与えられます。
この$N$回の対戦結果の集合の中には、同じ対戦の組合せが複数存在しても良いです。
この対戦結果の集合を用いて、$M$人のプレイヤの強さを推定したいです。

### 定式化
このような問題設定において、プレイヤの強さを推定するモデル・方法として、Bradley-Terry（BT）モデルが知られています。
BTモデルは、二人のプレイヤが対戦したときの勝率を計算するモデルです。
BTモデルでは、プレイヤ$i \in [M]$がプレイヤ$j \in [M]$に勝つ確率が以下で定義されます：
$$
    f_{\mathrm{BT}}(i \text{ beats } j; \mathbf{s}) = \sigma(s_i - s_j),
$$
ここで、$\mathbf{s} \in \mathbb{R}^{M}$はデータから推定・学習されるパラメータ、$\sigma: \mathbb{R} \to (0, 1)$はロジスティックシグモイド関数
$$
    \sigma(z) = \frac{1}{1 + \exp (-z)}
$$
です。
直観的には、$s_i \in \mathbb{R}$は$i$番目のプレイヤの強さを表しています：BTモデルでは$s_i - s_j$をロジスティックシグモイド関数に通すので、
 - $s_i - s_j$が大きければ大きいほどプレイヤ$i$がプレイヤ$j$に勝つ確率が高く、
 - $s_i - s_j$が小さければ小さいほどプレイヤ$i$がプレイヤ$j$に勝つ確率が低く、
 - $s_i = s_j$のとき、プレイヤ$i$がプレイヤ$j$に勝つ確率は$50$%と
 
推定されます。
BTモデルの気持ちが良い点として、プレイヤ$i$の勝率 = 1 - プレイヤ$j$の勝率が成り立つ、すなわち、
$$
    f_{\mathrm{BT}}(i \text{ beats } j; \mathbf{s}) = 1 - f_{\mathrm{BT}}(j \text{ beats } i; \mathbf{s})
$$
が成り立つことが挙げられます（=2人のプレイヤの勝率について矛盾が生じない。例えば、$i$と$j$が対戦について、$i$の勝率は90%、$j$の勝率は80%といったような勝率予測はされない）。

### BTモデルの推定
$M$人のプレイヤの間の、$N$回の対戦結果の集合
$$
    \mathcal{D} = \{ (i_n, j_n, y_n)\}_{n=1}^{N} \in \{[M] \times [M] \times \{0, 1\}\}^N
$$
が与えられたとします。ここで、$i_n, j_n \in [M]$は、$n \in [N]$ 番目の対戦を行ったプレイヤ、$y_n \in \{0, 1\}$は対戦結果を表します：$y_n=1$は$i_n$の勝ちを、$y_n=0$は$j_n$の勝ちを意味します。
$\mathcal{D}$が与えられたときのBTモデルの推定方法は様々考えられますが、例えば最尤推定を行う場合、$y_n$が$f_{\mathrm{BT}}(i \text{ beats } j; \mathbf{s})$を母数として持つベルヌーイ分布にしたがうと仮定して、以下の負の対数尤度を最小化すれば良いです：
$$
    -\sum_{n=1}^N \left[ y_n \log f_{\mathrm{BT}}(i_n \text{ beats } j_n; \mathbf{s}) + (1-y_n) \log f_{\mathrm{BT}}(j_n \text{ beats } i_n; \mathbf{s})\right].
$$
また、（$\mathbf{s}$の事前分布としてガウス分布を仮定した場合の）MAP推定では、上の式にさらに$\alpha \lVert \mathbf{s} \rVert_2^2 / 2$を加えたものを最小化すれば良いです（$\alpha > 0$は正則化項の強さを表すハイパーパラメータ）。

最尤推定・MAP推定の最適化問題はどちらも解析的には解けませんが、平滑で凸な関数の最小化問題であるため、勾配ベースの方法で効率よく解くことができます。

### BTモデルのロジスティック回帰としての表現
写像$x: [M] \times [M] \to \{0, 1, -1\}^M$を次のように定義します：
$$
    x(i, j)_k = \begin{cases}
    1 & \text{if } k=i,\\
    -1 & \text{if } k =j,\\
    0 & \text{otherwise}.
    \end{cases}
$$
このとき、BTモデルは以下のように書くことができます：
$$
    f_{\mathrm{BT}}(i \text{ beats } j; \mathbf{s}) = \sigma(s_i - s_j) = \sigma(\langle x(i, j), \mathbf{s}\rangle).
$$
ここで、$\langle \cdot , \cdot \rangle$は通常の内積です（２つのベクトルの要素ごとの積の和）。
すなわち、プレイヤ$i,j$に対するBTモデルは、$x(i,j) \in \{0, 1, -1\}^M$に対するロジスティック回帰と解釈できます。
したがって、BTモデルを推定したい場合、$\mathcal{D}$から$\mathcal{D}' = \{(x(i_n, j_n), y_n\}_{n=1}^N$を作り、$\mathcal{D}'$上でロジスティック回帰を推定すれば良いです（例えば`sklearn`を使う）。

### BTモデルの良いところ
プレイヤの強さの指標として、まず「勝率」を考える人も多いと思います。
すべてのプレイヤの組について、十分な数の対戦が行われていれば、勝率は良い指標となるでしょう。
しかし、実世界の問題においては、全てのプレイヤの組について十分な数の対戦が行われていない場合が多いでしょう。
また、より悪いことに、対戦が行われていないプレイヤの組が多数存在することも珍しくないはずです。
このような場合、各プレイヤの勝率は対戦相手の偏りに影響を受けます：熟練度の低いプレイヤとの対戦割合が多いプレイヤは強さが過大評価され、熟練度の高いプレイヤとの対戦割合が多いプレイヤは強さが過剰評価されてしまいます。
一方で、BTモデルでは、自身と対戦相手の強さ（と解釈ができる・に対応する）パラメータに応じて勝率が計算されます。
そのため、対戦回数が十分でなく、各プレイヤの対戦相手の熟練度・強さに偏りがある場合であっても、BTモデルはプレイヤの強さを上手く推定することができます。

### BTモデルの限界
BTモデルでは、各プレイヤは「強さ」に対応するスカラのみを持ちます。
強さに対応するパラメータの値が大きいプレイヤは勝ちやすく、小さいプレイヤは負けやすい、と推定されます。
したがって、BTモデルでは「プレイヤの相性」といった要素を考えることができません。
たとえば、「グーしか出さないプレイヤ1」「チョキしか出さないプレイヤ2」「パーしか出さないプレイヤ3」の3人にじゃんけんを行ってもらうとします。
このとき、十分な数の対戦が行われたとしても、「プレイヤ1はプレイヤ2に有利」「プレイヤ1はプレイヤ3に不利」といったことは推定できず、単に「全員同じくらいの強さ」と推定されます。
「相性」を考慮したい場合は、BTモデルではなくBlade-Chestモデルを用いると良いです。


## BTR<sup>4</sup>モデル：BTモデルの麻雀のための拡張
BTモデルは以下のようなゲームに使うことができるのでした：
 - 1対1で行い、
 - 対戦結果としては勝ち負けだけが与えられる。
 
一方で、麻雀は以下のような特徴を持ちます：
 - 4人で行い、
 - 対戦結果として「勝ち負け」だけでなく「点数」が与えられる、4人の点数の合計は0点である。
 
したがって、麻雀に対してBTモデルをそのまま使うことはできません。

そこで、BTモデルを拡張して麻雀に使えるようにします。
以下では、まず、対戦結果として勝ち負けだけでなく点数（スコア）が与えられる場合に拡張します。
そして次に、4人で行う場合に拡張します。

### 麻雀のための拡張1：対戦結果として点数が与えられる場合
対戦結果として、勝ち負けだけでなく、二人のプレイヤのスコアが与えられるとします。
ただし、二人のプレイヤの点数の合計は$0$であるとします。
前述の通り、BTモデルは、プレイヤ$i$がプレイヤ$j$に勝つ確率を、$\sigma(s_i - s_j)$と定義しており、また、$s_i, s_j$はそれぞれプレイヤ$i$と$j$の強さ（として解釈ができるの）でした。
そこで、BTモデルをこの問題設定に拡張したモデル、Bradley-Terry Regression（BTR）モデル$f_{\mathrm{BTR}}(\cdot, \cdot; \mathbf{s}): [M]^2 \to \mathbb{R}$を以下のように定義します：
$$
  f_{\mathrm{BTR}}(i, j; \mathbf{s}) = s_i - s_j.
$$
ここで、 $f_{\mathrm{BTR}}(i, j; \mathbf{s})$は、プレイヤ$i$とプレイヤ$j$が対戦したときのプレイヤ$i$のスコアです。 
当然、BTRモデルにおける、プレイヤ$i$とプレイヤ$j$が対戦したときのプレイヤ$j$のスコアは
$$
  f_{\mathrm{BTR}}(j, i; \mathbf{s}) =  s_j - s_i
$$
であり、この２つの点数の合計は明らかに$0$です（気持ち良い！）。

BTRモデルの推定は、例えば最尤推定を行う場合、「プレイヤ$i_n$とプレイヤ$j_n$が対戦した場合のプレイヤ$i_n$の点数$y_n\in \mathbb{R}$」は「$s_{i_n} - s_{j_n}$を平均とするガウス分布」にしたがうと仮定して、以下の負の対数尤度

$$
  \frac{1}{2} \sum_{n=1}^N \left(f_{\mathrm{BTR}}(i_n, j_n; \mathbf{s}) - y_n\right)_2^2 = \frac{1}{2} \sum_{n=1}^N \left(\left(s_{i_n} - s_{j_n}\right) - y_n\right)_2^2
$$
を最小化する等が考えられます（ガウス分布でなく他の分布でも良い、最適化できるかは分からないが）。
また、$s_i - s_j = \langle x(i, j), \mathbf{s}\rangle$であるので、前述のBTモデルがロジスティック回帰として解釈できるのと同様に、BTRモデルは線形回帰と解釈できます。

### 麻雀のための拡張2：4人で行う場合
プレイヤが4人である場合、対戦結果は$[M]^4 \times \mathbb{R}^4$の要素として表現されます。
この問題設定に対しては、BTRモデルを2入力1出力から4入力4出力に拡張、すなわち、$[M]^2 \to \mathbb{R}$から$[M]^4 \to \mathbb{R}^4$に拡張すれば良さそうです（スコアの和が0であるという制約の関係で、$[M]^4 \to \mathbb{R}^3$で良いですが、簡単のため、$[M]^4 \to \mathbb{R}^4$とします）。
もとのBTRモデルに対して、4出力に拡張されたBTRモデルをBTR<sup>4</sup>モデルと呼ぶことにします。

前述のBTRモデルでは、「自身の強さ -　相手の強さ」として、自身のスコアが定義されていました。
そこで、BTR<sup>4</sup>モデルでは、ある対戦における自分のスコアを、「（自身の強さ - 相手1の強さ） + （自身の強さ - 相手2の強さ） + （自身の強さ - 相手3の強さ）」と定義することにします。
フォーマルに書くと、BTR<sup>4</sup>モデルでは、プレイヤ$i_1, i_2, i_3, i_4\in [M]$が対戦した場合のプレイヤ$i_j$、$j \in [4]$のスコアを以下のように定義します：
$$
  f_{\mathrm{BTR}}^4(i_1, \ldots, i_4; \mathbf{s})_j = \sum_{k=1}^4 (s_j - s_k).
$$
例えば、プレイヤ1,2,4,6が対戦したときの、プレイヤ4のスコアは
$$
  f_{\mathrm{BTR}}^4({1,2,4,6}; \mathbf{s})_3 = (s_4 - s_1) + (s_4 - s_2) + (s_4 - s_6)
$$
となります。
この定義は対戦を4人で行う場合のものですが、より一般に、$K$人で行う場合に拡張すると以下のようになります：
$$
    f_{\mathrm{BTR}}^K(i_1, \ldots, i_K; \mathbf{s})_j = \sum_{k=1}^{K} (s_{i_j} - s_{i_k}).
$$
BTR$^2$モデルはBTRモデルと一致するため、この意味で、BTR$^K$モデルはBTRモデルの拡張となっています。

今、対戦結果の集合として、
$$
    \mathcal{D} = \{(i_{n,1}, i_{n,2}, i_{n,3}, i_{n,4}, y_{n, 1}, y_{n,2}, y_{n,3}, y_{n,4})\}_{n=1}^N
$$
が与えられたとします。
ここで、$i_{n, j}$は、$n \in [N]$番目の対戦結果における$j \in [4]$番目のプレイヤであり、$y_{n, j}$はそのマッチにおける$j$番目のプレイヤのスコアです。
このとき、BTR<sup>4</sup>モデルの推定は、例えば最尤推定を行う場合、以下の負の対数尤度
$$
  \frac{1}{2} \sum_{n=1}^N \sum_{j=1}^4 \left(f_{\mathrm{BTR}}^4(i_{n, 1}, \ldots, i_{n,4})_j- y_{n, j}\right)^2 = \frac{1}{2} \sum_{n=1}^N \sum_{j=1}^4 \left( \sum_{k=1}^4 \left(s_{n, i_j} - s_{n, i_k}\right) - y_{n, j}\right)^2
$$
を最小化すれば良いです。

また、写像$x': [M]^4 \to \{0, 1, -1\}^M$を次のように定義します：
$$
    x'(i_1, \{i_2, i_3, i_4\})_k = \begin{cases}
    3 & \text{if } k=i_1,\\
    -1 & \text{if } k \in \{i_2, i_3, i_4\},\\
    0 & \text{otherwise}.
    \end{cases}
$$
このとき、$f_{\mathrm{BTR}}^4(i_1, \ldots, i_4; \mathbf{s})_j = \sum_{k=1}^{4} (s_{i_j} - s_{i_k}) = \langle x'(i_j, \{i_1, \ldots, i_4\} \setminus \{i_j\}), \mathbf{s}\rangle$となります。
したがって、$N$個のデータ上でのBTR<sup>4</sup>モデルの推定は、（$4N$個の観測データ上の）線型回帰の推定とみなせます。

以上がBTモデルの麻雀のための拡張の大まかな説明になります。

### この拡張の気持ち悪い点
ある一回の対戦結果$(i_1, i_2, i_3, i_4, y_{i_1}, y_{i_2}, y_{i_3}, y_{i_4})$が与えられたとき、$y_{i_1}, y_{i_2}, y_{i_3}, y_{i_4}$はそれぞれ独立に観測されている、と暗に仮定しています（線型回帰との関係より）。
実際はそうではないでしょうから、この点は気持ち悪い・雑に感じます。

## 実際に推定する
長々と説明しましたが、実際に、Mリーグの2021/4/19までの対局結果のデータを用いて、Mリーガーの強さを推定してみます。

### データの読み込み

In [1]:
import numpy as np
from load_dataset import remove_ranking_point, add_ranking_point, load_mleague_dataset
from bradley_terry import BT
from blade_chest import BladeChestInner
from sklearn.model_selection import GridSearchCV

X, y = load_mleague_dataset("./data/")
y_original = remove_ranking_point(y) # 素点

print(X)
print(y)
print(y_original)
n_players = len(np.unique(X))
print(f"プレイヤ数：{n_players}")

[['佐々木寿人' '二階堂亜樹' '茅森早香' '黒沢咲']
 ['勝又健志' '佐々木寿人' '萩原聖人' '近藤誠一']
 ['萩原聖人' '滝沢和典' '朝倉康心' '魚谷侑未']
 ...
 ['堀慎吾' '佐々木寿人' '松本吉弘' '本田朋広']
 ['松ヶ瀬隆弥' '茅森早香' '石橋伸洋' '鈴木たろう']
 ['東城りお' '園田賢' '小林剛' '二階堂亜樹']]
[[101.7  -8.9 -30.3 -62.5]
 [ 77.   14.  -34.7 -56.3]
 [ 65.6   7.8 -16.9 -56.5]
 ...
 [ 60.9   9.6 -15.2 -55.3]
 [ 79.7  17.9 -32.2 -65.4]
 [ 55.9  10.2 -14.9 -51.2]]
[[ 56.7 -13.9 -15.3 -27.5]
 [ 32.    9.  -19.7 -21.3]
 [ 20.6   2.8  -1.9 -21.5]
 ...
 [ 15.9   4.6  -0.2 -20.3]
 [ 34.7  12.9 -17.2 -30.4]
 [ 10.9   5.2   0.1 -16.2]]
プレイヤ数：35


<a id='btr4_estimation_with_rank_point'></a>
### BTR<sup>4</sup>モデルの推定 - 順位点も込み

In [2]:
bt = BT(alpha=1e-6) # alpha：正則化項の強さ
bt.fit(X, y)
strength_sorted = np.sort(bt.coef_)[::-1]
players_sorted = bt.enc_.inverse_transform(np.argsort(bt.coef_)[::-1])

print("強さランキング")
print(players_sorted)
print("強さ")
print(strength_sorted)

強さランキング
['伊達朱里紗' '瑞原明奈' '堀慎吾' '多井隆晴' '松ヶ瀬隆弥' '小林剛' '東城りお' '沢崎誠' '鈴木たろう' '佐々木寿人'
 '近藤誠一' '滝沢和典' '勝又健志' '茅森早香' '黒沢咲' '白鳥翔' '内川幸太郎' '松本吉弘' '魚谷侑未' '朝倉康心'
 '日向藍子' '村上淳' '丸山奏子' '園田賢' '二階堂亜樹' '前原雄大' '岡田紗佳' '高宮まり' '藤崎智' '瀬戸熊直樹'
 '石橋伸洋' '本田朋広' '和久津晶' '萩原聖人' '二階堂瑠美']
強さ
[ 7.27734953  2.91537022  2.84834409  2.66620258  2.05428038  1.82700752
  1.69923057  1.55474089  1.33467458  1.12474554  1.07193666  1.03847351
  0.91722629  0.90723292  0.86006458  0.33081597  0.28782964  0.17765181
 -0.05495724 -0.1944098  -0.25807651 -0.42035695 -0.50047306 -0.63596502
 -0.81262952 -1.38569689 -1.8619187  -2.21135403 -2.23697281 -2.52202045
 -2.65969185 -3.66977638 -3.67989219 -3.84677984 -3.94220604]


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


上の結果からわかるように、「伊達朱里紗プロが一番強い」という推定結果になりました。

### BTR<sup>4</sup>モデルの推定 - 順位点抜き
BTR<sup>4</sup>モデルでは、プレイヤのスコアは「"自身の強さ - 相手の強さ"の和」として定義されています。
そのため、直観的には、順位点をそのまま使うのはやや不自然に思えます。
そこで、今度は素点を用いてプレイヤの強さを推定してみます。

In [3]:
bt = BT(alpha=1e-6)
bt.fit(X, y_original)
strength_sorted = np.sort(bt.coef_)[::-1]
players_sorted = bt.enc_.inverse_transform(np.argsort(bt.coef_)[::-1])

print("強さランキング")
print(players_sorted)
print("強さ")
print(strength_sorted)

強さランキング
['伊達朱里紗' '松ヶ瀬隆弥' '小林剛' '多井隆晴' '瑞原明奈' '東城りお' '堀慎吾' '沢崎誠' '滝沢和典' '佐々木寿人'
 '近藤誠一' '松本吉弘' '茅森早香' '勝又健志' '鈴木たろう' '黒沢咲' '朝倉康心' '魚谷侑未' '丸山奏子' '白鳥翔'
 '園田賢' '内川幸太郎' '二階堂亜樹' '岡田紗佳' '村上淳' '高宮まり' '日向藍子' '前原雄大' '瀬戸熊直樹' '藤崎智'
 '石橋伸洋' '萩原聖人' '本田朋広' '二階堂瑠美' '和久津晶']
強さ
[ 3.11342047  1.25463933  1.01791256  0.96085419  0.86436312  0.85718185
  0.78125165  0.63975156  0.60306714  0.58859253  0.41554047  0.29993871
  0.29335808  0.25431041  0.23735454  0.2009904   0.18981196  0.13990502
 -0.06647971 -0.09693573 -0.10005422 -0.11668202 -0.19563322 -0.379904
 -0.55468172 -0.59761078 -0.67895048 -0.7733082  -0.84385566 -0.89414172
 -1.14779072 -1.16863538 -1.54717329 -1.61133511 -1.93907203]


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


この場合でも、「伊達朱里紗プロが一番強い」という結果になりました。
2位以下は結果がいくらか変わっています。
例えば、松ヶ瀬隆弥プロや小林剛プロが順位を上げています。

## ベイズ信用区間

In [5]:
from bradley_terry import BayesianBT
bayesian_bt = BayesianBT(n_iter=1000, verbose=True)
bayesian_bt.fit(X, y_original)
strength_sorted_idx = np.argsort(bayesian_bt.coef_)[::-1]
strength_sorted = bayesian_bt.coef_[strength_sorted_idx]
strength_sigma_sorted = np.diagonal(bayesian_bt.sigma_)[strength_sorted_idx]
players_sorted = bt.enc_.inverse_transform(strength_sorted_idx)
print("強さランキング")
print(players_sorted)
print("強さ")
print(strength_sorted)
print("分散")
print(strength_sigma_sorted)

Convergence after  16  iterations
強さランキング
['伊達朱里紗' '小林剛' '多井隆晴' '松ヶ瀬隆弥' '滝沢和典' '瑞原明奈' '沢崎誠' '佐々木寿人' '堀慎吾' '近藤誠一'
 '東城りお' '松本吉弘' '茅森早香' '勝又健志' '朝倉康心' '鈴木たろう' '黒沢咲' '魚谷侑未' '丸山奏子' '白鳥翔'
 '内川幸太郎' '園田賢' '二階堂亜樹' '岡田紗佳' '藤崎智' '日向藍子' '高宮まり' '村上淳' '前原雄大' '二階堂瑠美'
 '瀬戸熊直樹' '本田朋広' '和久津晶' '石橋伸洋' '萩原聖人']
強さ
[ 0.82339537  0.61739934  0.59283554  0.44489591  0.40717138  0.39781678
  0.38572382  0.38036259  0.3776174   0.25429059  0.21900096  0.19969107
  0.16251592  0.14936001  0.11711207  0.11332056  0.09179027  0.08044914
 -0.0219178  -0.05015706 -0.0688574  -0.07292478 -0.12354697 -0.19431849
 -0.30320569 -0.32610605 -0.32932189 -0.38202696 -0.38605935 -0.44854277
 -0.57015976 -0.57359564 -0.60991603 -0.6190042  -0.73508789]
分散
[0.21877374 0.11465844 0.11136083 0.18999658 0.10865712 0.15667699
 0.12533348 0.10558287 0.14835891 0.13117453 0.22677575 0.11460609
 0.12193886 0.10993839 0.12481404 0.11252138 0.12095049 0.11821699
 0.20076619 0.11914669 0.13377796 0.1112964  0.12666853 0.16703555
 0.1944